In [1]:
import rospy
from franka_cal_sim.srv import*
import cv2
from cv_bridge import CvBridge
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, GRU, Masking, LSTM
from tensorflow.keras.layers import Add, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.compat.v1.keras import backend as K
import datetime
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tempfile import TemporaryFile

import random
from collections import deque

def stack_samples(samples):
    array = np.array(samples)

    current_states = np.stack(array[:,0]).reshape((array.shape[0],-1,array[0,0].shape[2]))
    current_act_hists = np.stack(array[:,1]).reshape((array.shape[0],-1,array[0,1].shape[2]))
    actions = np.stack(array[:,2]).reshape((array.shape[0],-1))
    rewards = np.stack(array[:,3]).reshape((array.shape[0],-1))
    new_states = np.stack(array[:,4]).reshape((array.shape[0],-1,array[0,4].shape[2]))
    new_act_hists = np.stack(array[:,5]).reshape((array.shape[0],-1,array[0,5].shape[2]))
    hiddens = np.stack(array[:,6]).reshape((array.shape[0],array[0,6].shape[1]))
    new_hiddens = np.stack(array[:,7]).reshape((array.shape[0],array[0,7].shape[1]))
    dones = np.stack(array[:,8]).reshape((array.shape[0],1))

    return current_states, current_act_hists, actions, rewards, new_states, new_act_hists, hiddens, new_hiddens, dones

In [2]:
class ModelBasedRL:
    def __init__(self, obs_dim, act_dim, sess):
        
        #data flow:
        #last_hidden, obs, last_act -> hidden_state
        #trans: hidden_state, act -> next_obs
        #reward: hidden_state, act -> reward
        #actor: hidden_state -> new act
        #critic: hidden_state, act -> Q
        
        ##session
        self.sess = sess
        
        ##hyperparameter:
        #learning rate
        self.trans_lr = 1e-4 
        self.reward_lr = 1e-4 
        self.actor_lr = 1e-4 
        self.critic_lr = 1e-4
        
        #training
        self.batch_size = 64   
        self.reg_w = 0.01
        
        #rl
        self.gamma = .99
        self.tau   = 0.01
        self.num_step = 3
        self.hidden_dim = 64
        
        #build models
        #h: hidden model
        #t: transition model
        #r: reward model
        #a: actor model
        #c: critic model
        self.obs_input, self.last_act_input,self.hidden_input,self.h_model = self.create_hidden_model()
        self.t_action_input,self.t_model = self.create_trans_model()
        self.r_action_input,self.r_model = self.create_reward_model()
        self.a_model = self.create_actor_model()
        self.c_model = self.create_critic_model()
        
        #target networks (mixed cases)
        self.tgt_a_model = self.create_actor_model()
        self.tgt_c_model = self.create_critic_model()
        
        #create actor loss
        self.init_s_input,self.init_a_input,self.init_h_input, self.a_loss = self.create_actor_loss()
        self.a_optimizer = tf.keras.optimizers.Adam(learning_rate=self.actor_lr)
        self.a_opt_op = self.a_optimizer.minimize(self.a_loss,self.a_model.trainable_weights)
        
    def create_hidden_model(self):
        # ========================================================================= #
        #state, last hidden -> hidden
        state_input = Input(shape=(None,self.obs_dim))
        act_input = Input(shape=(None,self.act_dim))
        hidden_input = Input(shape=(self.hidden_dim,))
        hidden_rnn,state_h = GRU(self.hidden_dim, return_state=True, initial_state=hidden_input)(Concatenate()([state_input,act_input]))
        
        h_model  = Model([self.obs_input,self.last_act_input,self.hidden_input],state_h)

        return state_input, act_input, hidden_input, h_model   
        
    def create_trans_model(self):
        
        #hidden,action -> next_obs
        
        t_action_input = Input(shape=self.act_dim)
        t_action_h1    = Dense(500)(t_action_input)

        t_merged    = Concatenate()([self.hidden_state, t_action_h1])
        t_merged_h1 = Dense(500, activation='relu')(t_merged)
        t_output = Dense(self.hidden_dim, activation='linear')(t_merged_h1)
        t_model  = Model([self.obs_input,self.last_act_input,self.hidden_input,t_action_input],t_output)

        adam  = Adam(lr=self.trans_lr)
        t_model.compile(loss="mse", optimizer=adam)
        
        return t_action_input, t_model
        
    
    def create_reward_model(self):
        
        #hidden,action -> reward
        
        r_action_input = Input(shape=self.act_dim)
        r_action_h1    = Dense(500)(r_action_input)

        r_merged    = Concatenate()([self.hidden_state, r_action_h1])
        r_merged_h1 = Dense(500, activation='relu')(r_merged)
        r_output = Dense(1, activation='linear')(r_merged_h1)
        r_model  = Model([self.obs_input,self.last_act_input,self.hidden_input,r_action_input],r_output)

        adam  = Adam(lr=self.reward_lr)
        r_model.compile(loss="mse", optimizer=adam)
        return r_action_input, r_model
        
        return t_model
    
    def create_actor_model(self):
        # ========================================================================= #
        #hidden -> action
        a_h = Dense(500, activation='relu')(self.hidden_state)
        a_output = Dense(self.act_dim, activation='tanh')(a_h)

        a_model = Model([self.obs_input,self.last_act_input,self.hidden_input], a_output)
        adam  = Adam(lr=self.actor_lr)
        a_model.compile(loss="mse", optimizer=adam)
        
        return a_model
    
    
    def create_critic_model(self):
        
        #hidden,action -> reward
        
        c_action_input = Input(shape=self.act_dim)
        c_action_h1    = Dense(500)(c_action_input)

        c_merged    = Concatenate()([self.hidden_state, c_action_h1])
        c_merged_h1 = Dense(500, activation='relu')(c_merged)
        c_output = Dense(1, activation='linear')(c_merged_h1)
        c_model  = Model([self.obs_input,self.last_act_input,self.hidden_input,c_action_input],c_output)

        adam  = Adam(lr=self.critic_lr)
        c_model.compile(loss="mse", optimizer=adam)
        
        return c_action_input, c_model
        
        
    def create_actor_loss(self):
        #initial state, action input (fixed)
        init_s_input = Input(shape=(None,self.obs_dim))
        init_a_input = Input(shape=(None,self.act_dim))
        init_h_input = Input(shape=(None,self.act_dim))
        loss = 0
        
        #first loop
        h_0 = self.h_model([init_s_input, init_a_input,init_h_input])
        a_0 = self.a_model([init_s_input, init_a_input,init_h_input])
        r_0 = self.r_model([init_s_input, init_a_input, init_h_input, a_0])
        o_1 = self.t_model([init_s_input, init_a_input, init_h_input,a_0])
    
        loss -= r_0
        
        #second loop
        h_1 = self.h_model([o_1, a_0, h_0])
        a_1 = self.a_model([o_1, a_0, h_0])
        r_1 = self.r_model([o_1, a_0, h_0, a_1])
        o_2 = self.t_model([o_1, a_0, h_0, a_1])
        
        loss -= r_1
        
        #third loop
        a_2 = self.a_model([o_2, a_1, h_1])
        r_2 = self.r_model([o_2, a_1, h_1,a_2])
        
        loss -= r_2
        
        return init_s_input,init_a_input,init_h_input, loss
    
    def train_trans(self,samples):
        cur_states, cur_act_hists, actions, rewards, new_states,new_act_hists,hiddens,new_hiddens _ =  stack_samples(samples)
        
        self.t_model.fit([cur_states,cur_act_hists,hiddens,actions],new_states)
        
    def train_reward(self,samples):
        cur_states, cur_act_hists, actions, rewards, new_states,new_act_hists, _ =  stack_samples(samples)
        self.r_model.fit([cur_states,cur_act_hists,hiddens,actions],rewards)
        
    def train_actor(self,samples):
        init_obs = np.zeros((self.obs_dim))
        init_act = np.zeros((self.act_dim))
        init_h = np.zeros((self.hidden_dim))
        
        self.a_opt_op.run(feed_dict={
            self.init_s_input:init_obs,
            self.init_a_input:init_act,
            self.init_h_input:init_h
        })
        
        
        

SyntaxError: invalid syntax (<ipython-input-2-a5462b1f7afd>, line 163)

In [ ]:
def read_replay(path,actor,obs_dim,action_dim):
    f = open(path, "r")
    obs, actions, rewards, next_obs, terminals = [],[],[],[],[]
    for line in f:
        cols = line.strip().split('\t')
        obs_s = cols[0].split(';')
        obs_t = []
        for obss in obs_s:
            obs_t.append([float(v) for v in obss.split(',')])
        obs_s_tp1 = cols[3].split(';')
        obs_tp1 = []
        for obss in obs_s_tp1:
            obs_tp1.append([float(v) for v in obss.split(',')])
        obs_t = np.array(obs_t)
        obs.append(obs_t)
        action = np.array([float(v) for v in cols[1].split(',')])
        actions.append(action)
        rewards.append(float(cols[2]))
        obs_tp1 = np.array(obs_tp1)
        next_obs.append(obs_tp1)
        terminals.append(bool(cols[4]=="True"))

    print(terminals)
    i = np.argmax(rewards)
    print(i)
    print(obs[i])
    print(actions[i])
    print(rewards[i])
    plt.plot(np.linalg.norm(actions,axis=1))
    plt.show()
    plt.plot(rewards)
    plt.show()

    #save data
    for i in range(len(obs)):
        obs_action_seq = obs[i]
        obs_seq = obs_action_seq[:,0:12]
        ########
        #obs_seq = np.zeros((6,12))
        act_seq = obs_action_seq[:,12:48]
        #print(obs_seq)
        #print(act_seq)
        num_l = np.sum(abs(actions[i])>0.018)
        action = actions[i]
        if np.linalg.norm(action)<1:
            action = actions[i]/0.02



        reward = rewards[i]
        #reward-=num_l*0.5

        next_obs_action_seq = next_obs[i]
        next_obs_seq = next_obs_action_seq[:,0:12]
        next_act_seq = next_obs_action_seq[:,12:48]
        obs_seq = obs_seq.reshape((1, -1, obs_dim))
        act_seq = act_seq.reshape((1, -1, action_dim))
        #next_obs_seq = np.zeros((6,12))
        next_obs_seq = next_obs_seq.reshape((1, -1, obs_dim))
        ########
        next_act_seq = next_act_seq.reshape((1, -1, action_dim))
        done = terminals[i]
        actor.remember(obs_seq, act_seq, action, reward, next_obs_seq, next_act_seq, done)
        #augment data
        next_act_seq_1 = np.concatenate([act_seq.reshape(((-1,action_dim)))[1:],-action.reshape((1,action_dim))],axis = 0).reshape((1, -1, action_dim))
        next_act_seq_2 = np.concatenate([-act_seq.reshape(((-1,action_dim)))[1:],action.reshape((1,action_dim))],axis = 0).reshape((1, -1, action_dim))
        actor.remember(obs_seq, -act_seq, -action, reward, next_obs_seq, -next_act_seq, done)
        actor.remember(obs_seq, act_seq, -action, reward, next_obs_seq, next_act_seq_1, done)
        actor.remember(obs_seq, -act_seq, action, reward, next_obs_seq, next_act_seq_2, done)

    print(len(obs))


def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

def step(state, action):
    rospy.wait_for_service('/model_client/rl_service')
    rospy.loginfo(rospy.get_caller_id() + 'begin service')
    take_step = rospy.ServiceProxy('/model_client/rl_service', RLSrv)
    resp1 = take_step(0, state.tolist(), action.tolist())
    return resp1.next_state, resp1.reward, resp1.done

def reset():
    rospy.wait_for_service('/model_client/rl_service')
    rospy.loginfo(rospy.get_caller_id() + 'begin service')
    reset_step = rospy.ServiceProxy('/model_client/rl_service', RLSrv)
    resp1 = reset_step(1, [], [])
    return resp1.next_state, resp1.reward, resp1.done

In [ ]:
def simulation():
    #record the reward trend
    reward_list = []
    test_reward_list = []

    #save path:
    # actor_checkpoint = rospy.get_param("/rl_client/actor_checkpoint")
    # critic_checkpoint = rospy.get_param("/rl_client/critic_checkpoint")
    # fig_path = rospy.get_param("/rl_client/figure_path")
    # result_path = rospy.get_param("/rl_client/result_path")
    # test_result_path = rospy.get_param("/rl_client/test_result_path")

    #init
    obs_dim = 12
    action_dim = 36
    sess = tf.compat.v1.Session()
    K.set_session(sess)
    actor_critic = ActorCritic(sess)

    path = "/home/yunke/prl_proj/panda_ws/src/franka_cal_sim/python/replay_buffers/replay_buffer_imu_final.txt"
    #read replay buffer
    read_replay(path,actor_critic,obs_dim,action_dim)

    path = "/home/yunke/prl_proj/panda_ws/src/franka_cal_sim/python/replay_buffer_last.txt"
    #read replay buffer
    read_replay(path,actor_critic,obs_dim,action_dim)



    path = "/home/yunke/prl_proj/panda_ws/src/franka_cal_sim/python/replay_buffers/replay_buffer_tiny.txt"

    read_replay(path,actor_critic,obs_dim,action_dim)


    num_trials = 10000
    trial_len  = 3

    for i in range(num_trials):

        # reset()
        # cur_state = np.ones(obs_dim)*500
        # reward_sum = 0
        # obs_list = []
        # act_list = []
        # cur_state = cur_state.reshape((1,obs_dim))
        # obs_list.append(normalize(cur_state))
        # act_list.append(normalize(0.01*np.ones((1,action_dim))))
        # for j in range(trial_len):
        #     #env.render()
        #     print("trial:" + str(i))
        #     print("step:" + str(j))
        #     obs_seq = np.asarray(obs_list)
        #     print("obs_seq"+str(obs_seq))
        #     act_seq = np.asarray(act_list)
        #     obs_seq = obs_seq.reshape((1, -1, obs_dim))
        #     act_seq = act_seq.reshape((1, -1, action_dim))
        #     action = actor_critic.act(obs_seq,act_seq)
        #     action = action.reshape((action_dim))
        #     cur_state = cur_state.reshape(obs_dim)
        #     new_state, reward, done = step(cur_state,action)
        #     reward_sum += reward
        #     rospy.loginfo(rospy.get_caller_id() + 'got reward %s',reward)
        #     if j == (trial_len - 1):
        #         done = True



            actor_critic.train(i)
            actor_critic.update_target()
            obs_list = []
            act_list = []
            cur_state = np.zeros(obs_dim)
            cur_state = cur_state.reshape((1,obs_dim))
            obs_list.append(cur_state)
            act_list.append(0.01*np.ones((1,action_dim)))
            obs_seq = np.asarray(obs_list)
            act_seq = np.asarray(act_list)
            obs_seq = obs_seq.reshape((1, -1, obs_dim))
            act_seq = act_seq.reshape((1, -1, action_dim))
            action = actor_critic.actor_model.predict([obs_seq,act_seq])*0.02
            if i%200==0:
                print(action[0])
                print(np.linalg.norm(action[0]))
                action = np.ones((1,action_dim))
                print(actor_critic.critic_model.predict([obs_seq,act_seq,action]))
                action = np.ones((1,action_dim))*0.05
                act_seq = np.ones((1,1,action_dim))*0.001
                print(actor_critic.critic_model.predict([obs_seq,act_seq,action]))

            # new_state = np.asarray(new_state).reshape((1,obs_dim))
            # action = action.reshape((1,action_dim))

            # obs_list.append(normalize(new_state))
            # act_list.append(normalize(action))
            # next_obs_seq = np.asarray(obs_list)
            # next_act_seq = np.asarray(act_list)
            # next_obs_seq = next_obs_seq.reshape((1, -1, obs_dim))
            # next_act_seq = next_act_seq.reshape((1, -1, action_dim))

            # #padding
            # pad_width = trial_len-np.size(obs_seq,1)
            # rospy.loginfo(rospy.get_caller_id() + 'obs_shape %s',obs_seq.shape)
            # obs_seq = np.pad(obs_seq,((0,0),(pad_width,0),(0,0)),'constant')
            # next_obs_seq = np.pad(next_obs_seq,((0,0),(pad_width,0),(0,0)),'constant')
            # act_seq = np.pad(act_seq,((0,0),(pad_width,0),(0,0)),'constant')
            # next_act_seq = np.pad(next_act_seq,((0,0),(pad_width,0),(0,0)),'constant')
            # #print(obs_seq.shape)
            # #print(next_obs_seq.shape)

            # actor_critic.remember(obs_seq, act_seq, action, reward, next_obs_seq, next_act_seq, done)
            # cur_state = new_state
            # if done:
            #     rospy.loginfo(rospy.get_caller_id() + 'got total reward %s',reward_sum)
            #     reward_list.append(reward_sum)
            #     break

        # if i % 5 == 0:
        #     actor_critic.actor_model.save_weights(actor_checkpoint)
        #     actor_critic.critic_model.save_weights(critic_checkpoint)
        #     fig, ax = plt.subplots()
        #     ax.plot(reward_list)
        #     fig.savefig(fig_path)
        #     np.savetxt(result_path, reward_list, fmt='%f')
        #
    #draw a graph
    obs_list = []
    act_list = []
    cur_state = np.zeros(obs_dim)
    cur_state = cur_state.reshape((1,obs_dim))
    obs_list.append(cur_state)
    act_list.append(0.01*np.ones((1,action_dim)))
    obs_seq = np.asarray(obs_list)
    act_seq = np.asarray(act_list)
    obs_seq = obs_seq.reshape((1, -1, obs_dim))
    act_seq = act_seq.reshape((1, -1, action_dim))
    q = []
    t_q = []
    for i in range(100):
        action = 0.015*np.ones((1,action_dim))*i
        q_value = actor_critic.critic_model.predict([obs_seq,act_seq,action])
        t_q_value = actor_critic.target_critic_model.predict([obs_seq,act_seq,action])
        t_q.append(t_q_value.reshape((-1,)))
        q.append(q_value.reshape((-1,)))

    plt.plot(q)
    plt.show()
    plt.plot(t_q)
    plt.show()
    # #test
    # rospy.init_node('RL_client', anonymous=True)
    # reset()
    # cur_state = np.zeros(obs_dim)
    # test_reward_sum = 0
    # test_reward_sum = 0
    # obs_list = []
    # act_list = []
    # cur_state = cur_state.reshape((1,obs_dim))
    # obs_list.append(cur_state)
    # act_list.append(0.01*np.ones((1,action_dim)))
    # for j in range(trial_len):
    #     #env.render()
    #     print("test:" + str(i))
    #     print("step:" + str(j))
    #
    #     obs_seq = np.asarray(obs_list)
    #     print("obs_seq:" + str(obs_seq))
    #     act_seq = np.asarray(act_list)
    #     #obs_seq = np.zeros((6,12))
    #     obs_seq = obs_seq.reshape((1, -1, obs_dim))
    #
    #     act_seq = act_seq.reshape((1, -1, action_dim))
    #     action = actor_critic.actor_model.predict([obs_seq,act_seq])*0.02
    #     action = action.reshape((action_dim))
    #     print(action)
    #     cur_state = cur_state.reshape(obs_dim)
    #     new_state, reward, done = step(cur_state,action)
    #     test_reward_sum += reward
    #     rospy.loginfo(rospy.get_caller_id() + 'got reward %s',reward)
    #     if j == (trial_len - 1):
    #         done = True
    #
    #
    #     new_state = np.asarray(new_state).reshape((1,obs_dim))
    #     action = action.reshape((1,action_dim))
    #
    #     obs_list.append(new_state)
    #     act_list.append(action)
    #     cur_state = new_state
    #     if done:
    #         rospy.loginfo(rospy.get_caller_id() + 'got total reward %s',test_reward_sum)
    #         test_reward_list.append(test_reward_sum)
    #         # fig1, ax1 = plt.subplots()
    #         # ax1.plot(test_reward_list)
    #         #fig1.savefig('/home/yunke/prl_proj/panda_ws/src/franka_cal_sim/python/test_reward.png')
    #         #np.savetxt(test_result_path, test_reward_list, fmt='%f')
    #         break



if __name__ == '__main__':
    simulation()
